In [1]:
%env CUDA_VISIBLE_DEVICES=0

device='cuda'

env: CUDA_VISIBLE_DEVICES=0


In [2]:
if False:
    !pip install mtcnn
    !pip install timm==0.4.5
    !pip install tensorflow
    !pip install numba
    #!pip install keras==2.4.3
    #!pip install -q torch==1.7.1 torchvision
    !pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from copy import deepcopy
from tqdm import tqdm

from mtcnn import MTCNN

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model, model_from_json
from tensorflow.keras.optimizers import Adam, SGD

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms

print(f"Torch: {torch.__version__}")

2024-04-11 08:44:19.404419: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 08:44:19.441744: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 08:44:19.441768: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 08:44:19.441798: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 08:44:19.449513: I tensorflow/core/platform/cpu_feature_g

Torch: 2.1.0+cu121


In [4]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense, Activation, Concatenate, Reshape
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax

In [5]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Reserved:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA A100-PCIE-40GB
Memory Usage:
Allocated: 0.0 GB
Reserved:    0.0 GB


In [6]:
CONVERT2INT=True
if CONVERT2INT is True:
    N_CLASSES = 2
else:
    N_CLASSES = 4

#### Helpers

In [7]:
import pickle

def save_features(file_name, features):
    if os.path.isfile(file_name):
        print("Error! Cannot save features because file already exists")
        return
    with open(file_name, 'wb') as f:
        pickle.dump(features, f) # , protocol=pickle.HIGHEST_PROTOCOL)
    
def load_features(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

def save_weights(model, file_name):
    if os.path.isfile(file_name):
        print("Error! Cannot save features because file already exists")
        return
    model.save_weights(file_name)

def load_weights(model, file_name):
    model.load_weights(file_name)

In [8]:
test_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

def extract_features(root_dir, directories):
    all_features = []
    for i, d in enumerate(directories):
        if d.startswith('.'):
            continue
        print('[{}/{}]'.format(i+1, len(directories)), end="\r") 
        imgs = []
        sequence = []
        path = os.path.join(root_dir, d)
        for img in os.listdir(path):
            if not img.endswith(ext):
                continue
            img_file = os.path.join(path, img)
            if torch_model:
                image = Image.open(img_file)
                img_tensor = test_transforms(image)
                imgs.append(img_tensor)
            else:
                image = cv2.imread(img_file)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                imgs.append(image)
            
            if len(imgs) >= BATCH_SIZE:
                if torch_model:
                    features = cnn_model(torch.stack(imgs, dim=0).to(device))
                    features = features.data.cpu().numpy()
                else:
                    features = cnn_model.predict(np.array(imgs), verbose=0)
                if len(sequence) == 0:
                    sequence = features
                else:
                    sequence = np.concatenate((sequence, features),axis=0)
                imgs = []
                
        if len(imgs) > 0:        
            if torch_model:
                features = cnn_model(torch.stack(imgs, dim=0).to(device))
                features = features.data.cpu().numpy()
            else:
                features = cnn_model.predict(np.array(imgs), verbose=0)
            if len(sequence) == 0:
                sequence = features
            else:
                sequence = np.concatenate((sequence, features),axis=0)
        
        all_features.append(sequence)
    return all_features

def extract_or_read_features(train_pickle, test_pickle, train_data_files, test_data_files):
    if os.path.isfile(train_pickle):
        train_features, train_data_files = load_features(train_pickle)
    else:
        print("Create train dataset")
        train_features = extract_features(train_dir, train_data_files)
        save_features(train_pickle, [train_features, train_data_files])
    if os.path.isfile(test_pickle):
        test_features, test_data_files = load_features(test_pickle)
    else:
        print("Create test dataset")
        test_features = extract_features(val_dir, test_data_files)
        save_features(test_pickle, [test_features, test_data_files])
    return train_features, train_data_files, test_features, test_data_files

In [9]:
def create_file2features(files, features):
    res = {}
    for i in range(len(files)):
        res[files[i]] = features[i]
    return res

## Prepare dataset

In [10]:
DATA_DIR='/home/HDD6TB/datasets/emotions/EmotiW/engagement/'
ext = "png"

In [17]:
!pwd

/home/echuraev/Workspace/HSE/engagement


In [10]:
import csv
labels_list = ['distracted', 'engaged']
video2label={}
float2int={0:0,0.33:1,0.66:2,1:3}
float2int={0:0,0.33:0,0.66:1,1:1}

import csv
video2label={}
with open(os.path.join(DATA_DIR,'Engagement_Labels_Engagement.csv'), mode='r') as csvfile:
    labels_reader = csv.reader(csvfile, delimiter='\t')
    for i,row in enumerate(labels_reader):
        if i==0:
            print('first:',row)
            continue
        videoname,label=row[0],float(row[1])
        video2label[videoname]=label
        #print(videoname,label)
        #if (videoname not in filename2features_val) and (videoname not in filename2features_train):
        #    print(videoname,label)
#check if fix is incorrect
video2label['subject_87_Vid_3']=video2label['subject_77_Vid_6']
#video2label = to_categorical(video2label)
print(len(video2label))
print(video2label)

FileNotFoundError: [Errno 2] No such file or directory: '/home/HDD6TB/datasets/emotions/EmotiW/engagement/Engagement_Labels_Engagement.csv'

In [16]:
INPUT_SIZE = (224, 224)
model_urls = {
    #'affectnet_7_vggface2_rexnet150.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/affectnet_7_vggface2_rexnet150.pt',
    # 'enet_b0_7.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b0_7.pt',
    'enet_b0_8_best_afew.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b0_8_best_afew.pt',
    # 'enet_b0_8_best_vgaf.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b0_8_best_vgaf.pt',
    # 'enet_b2_7.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b2_7.pt',
    # 'enet_b2_8.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/enet_b2_8.pt',
    'mobilenet_7.h5': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/b55d78c7d2a4c02339de916936fc73749eb58798/models/affectnet_emotions/mobilenet_7.h5',
    # 'enet_b0_8_va_mtl.pt': 'https://github.com/HSE-asavchenko/face-emotion-recognition/raw/24c7b22228f88429bf11c64f1f3f292b8e8abe32/models/affectnet_emotions/enet_b0_8_va_mtl.pt',
}

def download_model(model_url, model_name):
    import urllib.request

    if not os.path.isfile(model_name):
        urllib.request.urlretrieve(model_url, model_name)

In [17]:
FEATURES_DIR = "features/"
WEIGHTS_DIR = 'weights/'

In [18]:
def print_acc(model,filename2features):
    num_correct,num_total=0,0
    for fn in filename2features:
        features=filename2features[fn]
        total_features=None
        if USE_ALL_FEATURES:
            x=features[0][features[-1]==1]
        else:
            x=features
        max_ind=len(x)-IMAGE_SET_SIZE
        if max_ind<=0:
            continue
        if CONCATENATE_STAT:
            stat_x=np.repeat(stat_function(x,axis=0).reshape((1,-1)),len(x),axis=0)
            inp = np.expand_dims(np.concatenate((stat_x,x),axis=1), axis=0)
            preds=model.predict(inp, verbose=0)[0]
        else:
            preds=model.predict(np.expand_dims(x, axis=0), verbose=0)[0]
        if np.argmax(preds) == float2int[video2label[fn]]:
            num_correct+=1
        num_total+=1
    print(num_total,num_correct/num_total)
    
def print_acc(model, test_features, test_labels):
    num_correct,num_total=0,0
    res_preds = []
    res_labels = []
    for i in range(len(test_features)):
        features=test_features[i]
        total_features=None
        if USE_ALL_FEATURES:
            x=features[0][features[-1]==1]
        else:
            x=features
        max_ind=len(x)-IMAGE_SET_SIZE
        if max_ind<=0:
            continue
        if CONCATENATE_STAT:
            stat_x=np.repeat(stat_function(x,axis=0).reshape((1,-1)),len(x),axis=0)
            inp = np.expand_dims(np.concatenate((stat_x,x),axis=1), axis=0)
            preds=model.predict(inp, verbose=0)[0]
        else:
            preds=model.predict(np.expand_dims(x, axis=0), verbose=0)[0]
        res_preds.append(np.argmax(preds))
        res_labels.append(test_labels[i])
        if res_preds[-1] == test_labels[i]:
            num_correct+=1
        num_total+=1
    acc = num_correct/num_total
    print(num_total, acc)
    return acc, res_preds, res_labels

### Facial images

In [15]:
FACES_DIR = os.path.join(DATA_DIR, "frames/faces/mtcnn_aligned")

In [16]:
train_dir = os.path.join(FACES_DIR, "Train")
val_dir = os.path.join(FACES_DIR, "validation")
train_data_files = os.listdir(train_dir)
test_data_files = os.listdir(val_dir)
print(len(train_data_files))
print(len(test_data_files))

147
48


In [17]:
def print_classes(files):
    labels = []
    for f in files:
        labels.append(float2int[video2label[f]])
    min_num = len(files)
    for i in range(N_CLASSES):
        count = labels.count(i)
        if count < min_num:
            min_num = count
        percent = count * 100 / len(labels)
        print("{} {}/{}: {}%".format(i, count, len(labels), percent))
    return min_num

print("Train:")
train_min_num = print_classes(train_data_files)
print("Test:")
test_min_num = print_classes(test_data_files)

Train:
0 40/147: 27.210884353741495%
1 107/147: 72.78911564625851%
Test:
0 14/48: 29.166666666666668%
1 34/48: 70.83333333333333%


In [17]:
def adjust_classes(files, min_num):
    tmp_files = []
    for i in range(N_CLASSES):
        filtered_files = []
        for j in range(len(files)):
            if len(filtered_files) == min_num:
                break
            label = float2int[video2label[files[j]]]
            if label == i:
                filtered_files.append(files[j])
        tmp_files.extend(filtered_files)
        
    res_files = []
    indices = list(range(len(tmp_files)))
    random.seed(7)
    random.shuffle(indices)
    for idx in indices:
        res_files.append(tmp_files[idx])
    
    return res_files

train_data_files = adjust_classes(train_data_files, train_min_num)
print(len(train_data_files))
test_data_files = adjust_classes(test_data_files, test_min_num)
print(len(test_data_files))

80
28


In [18]:
print("Train:")
print_classes(train_data_files)
print("Test:")
print_classes(test_data_files)

Train:
0 40/80: 50.0%
1 40/80: 50.0%
Test:
0 14/28: 50.0%
1 14/28: 50.0%


14

In [18]:
def write_or_read_file_names(pickle_name, train_data_files, test_data_files):
    if os.path.isfile(pickle_name):
        return load_features(pickle_name)
    else:
        save_features(pickle_name, [train_data_files, test_data_files])
    return train_data_files, test_data_files

train_data_files, test_data_files = write_or_read_file_names("ajusted_classes.pickle", train_data_files, test_data_files)

In [19]:
SPLIT_RATIO = len(train_data_files) / (len(train_data_files) + len(test_data_files))
print(SPLIT_RATIO)

0.7407407407407407


## Prepare custom dataset

In [19]:
USERS_DATASET = "/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/"

DISTRACTED_DIR = os.path.join(USERS_DATASET, "distracted/")
ENGAGED_DIR = os.path.join(USERS_DATASET, "engaged/")
DISTRACTED_DIR_FACES = os.path.join(USERS_DATASET, "distracted_faces/")
ENGAGED_DIR_FACES = os.path.join(USERS_DATASET, "engaged_faces/")

In [20]:
def get_total_frames(video_file):
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    return int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

def get_video_frames(video_file, n_frames=1, frame_processor=None):
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))//n_frames
    for frame_count in range(n_frames,n_frames*(total_frames+1),n_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES,frame_count)
        ret, frame_bgr = cap.read()
        if not ret:
            cap.release()
            break
        frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        if frame_processor is not None:
            frame = frame_processor(frame)
        yield frame

def preprocess(frame):
    from functools import wraps
    import sys
    import io
    def capture_output(func):
        """Wrapper to capture print output."""

        @wraps(func)
        def wrapper(*args, **kwargs):
            old_stdout = sys.stdout
            new_stdout = io.StringIO()
            sys.stdout = new_stdout
            try:
                return func(*args, **kwargs)
            finally:
                sys.stdout = old_stdout

        return wrapper
    
    detector = MTCNN()
    w_detect_face = capture_output(detector.detect_faces)
    detected = w_detect_face(frame)
    if len(detected) == 0:
        return None
    face = detected[0]
    x1, y1, w, h = face['box'][0:4]
    x2 = x1 + w
    y2 = y1 + h
    frame = frame[y1:y2, x1:x2, :]
    return cv2.resize(frame, INPUT_SIZE)

def process_videos(video_dir, result_dir):
    for vid in os.listdir(video_dir):
        vid_path = os.path.join(video_dir, vid)
        out_path = os.path.join(result_dir, vid.split('.')[0])
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        else:
            print("Skipping: ", vid)
            continue
        total_frames = get_total_frames(vid_path)
        frames = get_video_frames(vid_path, frame_processor=preprocess)
        for i, f in enumerate(frames):
            print("[{}: {}/{}] ".format(vid, i, total_frames), end='\r')
            if f is None:
                continue
            cv2.imwrite(os.path.join(out_path, str(i) + '.png'), cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
        print()

In [20]:
process_videos(DISTRACTED_DIR, DISTRACTED_DIR_FACES)
process_videos(ENGAGED_DIR, ENGAGED_DIR_FACES)

FileNotFoundError: [Errno 2] No such file or directory: '/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/distracted/'

### Extract all features

In [15]:
path = '/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Egor_engaged_10/'
print(len(os.listdir(path)))

145


In [21]:
def get_frames_paths(directory):
    video2frames = {}
    for video_dir in os.listdir(directory):
        print("Dir: ", video_dir)
        path = os.path.join(directory, video_dir)
        total_files = len(os.listdir(path))
        video2frames[video_dir] = []
        i = 0
        while i < total_files:
            file_name = str(i) + ".png"
            file_path = os.path.join(path, file_name)
            i += 1
            if not os.path.exists(file_path):
                print(file_path)
                continue
            video2frames[video_dir].append(file_path)
    return video2frames

engaged2frames = get_frames_paths(ENGAGED_DIR_FACES)
distracted2frames = get_frames_paths(DISTRACTED_DIR_FACES)

Dir:  Yana_engaged_2
Dir:  Katya 16 engaged
Dir:  Den_engaged_5
Dir:  Egor_engaged_4
Dir:  Katya 1 engaged
Dir:  Egor_engaged_13
Dir:  Egor_engaged_3
Dir:  Yana_engaged_10
Dir:  Den_engaged_10
Dir:  Kate_engaged_9
Dir:  Den_engaged_7
Dir:  Katya 12 engaged
Dir:  Kate_engaged_6
Dir:  Yana_engaged_5
Dir:  Yana_engaged_6
Dir:  Den_engaged_2
Dir:  Kate_engaged_16
Dir:  Kate_engaged_4
Dir:  Katya 14 engaged
Dir:  Egor_engaged_16
Dir:  Den_engaged_4
Dir:  Den_engaged_14
Dir:  Katya 9 engaged
Dir:  Kate_engaged_7
Dir:  Katya 10 engaged
Dir:  Yana_engaged_14
Dir:  Kate_engaged_1
/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Kate_engaged_1/0.png
/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Kate_engaged_1/1.png
/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Kate_engaged_1/2.png
/home/echuraev/Workspace/HSE/datasets/Engagement_dataset/engaged_faces/Kate_engaged_1/3.png
Dir:  Yana_engaged_7
Dir:  Yana_engaged_16
Dir:  Egor_

In [22]:
test_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

def extract_features_my_ds(vid2frame, label):
    vid2features = {}
    #import re
    for j, k in enumerate(vid2frame.keys()):
        print('[{}/{}]'.format(j+1, len(vid2frame)), end="\r")
        #actor_name = re.split(" |_", k)[0]
        vid2features[k] = {}
        imgs = []
        sequence = []
        for i, vid in enumerate(vid2frame[k]):
            img_file = vid
            if torch_model:
                image = Image.open(img_file)
                img_tensor = test_transforms(image)
                imgs.append(img_tensor)
            else:
                image = cv2.imread(img_file)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                imgs.append(image)
            
            if len(imgs) >= BATCH_SIZE:
                if torch_model:
                    features = cnn_model(torch.stack(imgs, dim=0).to(device))
                    features = features.data.cpu().numpy()
                else:
                    features = cnn_model.predict(np.array(imgs), verbose=0)
                if len(sequence) == 0:
                    sequence = features
                else:
                    sequence = np.concatenate((sequence, features),axis=0)
                imgs = []  
        if len(imgs) > 0:        
            if torch_model:
                features = cnn_model(torch.stack(imgs, dim=0).to(device))
                features = features.data.cpu().numpy()
            else:
                features = cnn_model.predict(np.array(imgs), verbose=0)
            if len(sequence) == 0:
                sequence = features
            else:
                sequence = np.concatenate((sequence, features),axis=0)
        
        vid2features[k]['features'] = sequence
        vid2features[k]['label'] = label 
    print()
    return vid2features

def extract_or_read_features_my_ds(file_pickle, vid2frame = None, label: int = -1):
    if os.path.isfile(file_pickle):
        vid2features = load_features(file_pickle)
    elif vid2frame is None or label < 0:
        raise Exception("Cannot read file: ", file_pickle)
    else:
        vid2features = extract_features_my_ds(vid2frame, label)
        save_features(file_pickle, vid2features)
    return vid2features

In [23]:
for base_model_key in model_urls.keys():
    print("Model: ", base_model_key)
    download_model(model_urls[base_model_key], base_model_key)
    base_model_name = './' + base_model_key
    torch_model = True if base_model_name[base_model_name.rfind('.') + 1:] == 'pt' else False
    if torch_model:
        BATCH_SIZE = 64
    else:
        BATCH_SIZE = 128
        
    if base_model_key in ('enet_b2_7.pt', 'enet_b2_8.pt'):
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier,torch.nn.Softmax(dim=0))
        if True:
            inp = torch.randn(1408).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
        #print(cnn_model)
    elif base_model_key in ('enet_b0_8_va_mtl.pt'):
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier,torch.nn.Softmax(dim=1))
        if True:
            inp = torch.randn(20, 1280).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
        #print(cnn_model)
    elif torch_model:
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier[0],torch.nn.Softmax(dim=1))
        if True:
            inp = torch.randn(20, 1280).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
    else:
        base_model = load_model(base_model_name)
        layer_out = base_model.get_layer('global_pooling')
        cnn_model = Model(base_model.input,layer_out.output)

        #cnn_model.summary()
    
    engaged_pickle = USERS_DATASET + 'engaged_features_{}.experiments.pickle'.format(base_model_key)
    distracted_pickle = USERS_DATASET + 'distracted_features_{}.experiments.pickle'.format(base_model_key)
    print("Engaged:")
    _ = extract_or_read_features_my_ds(engaged_pickle, engaged2frames, 1)
    print("Distracted:")
    _ = extract_or_read_features_my_ds(distracted_pickle, distracted2frames, 0)

Model:  enet_b0_8_best_afew.pt
Engaged:
[80/80]
Distracted:
[80/80]
Model:  mobilenet_7.h5


2024-04-11 08:47:04.065703: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 32509 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Engaged:


2024-04-11 08:47:05.473873: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902


[80/80]
Distracted:
[80/80]


## Extract all features

In [26]:
for base_model_key in model_urls.keys():
    print("Model: ", base_model_key)
    download_model(model_urls[base_model_key], base_model_key)
    base_model_name = './' + base_model_key
    torch_model = True if base_model_name[base_model_name.rfind('.') + 1:] == 'pt' else False
    if torch_model:
        BATCH_SIZE = 64
    else:
        BATCH_SIZE = 128
        
    if base_model_key in ('enet_b2_7.pt', 'enet_b2_8.pt'):
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier,torch.nn.Softmax(dim=0))
        if True:
            inp = torch.randn(1408).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
        #print(cnn_model)
    elif base_model_key in ('enet_b0_8_va_mtl.pt'):
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier,torch.nn.Softmax(dim=1))
        if True:
            inp = torch.randn(20, 1280).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
        #print(cnn_model)
    elif torch_model:
        cnn_model = torch.load(base_model_name)
        cnn_model = cnn_model.to(device)
        cnn_model.eval()

        #del cnn_model.classifier
        last_layer=torch.nn.Sequential(cnn_model.classifier[0],torch.nn.Softmax(dim=1))
        if True:
            inp = torch.randn(20, 1280).to(device)
            f=last_layer.forward(inp)
            #print(f.shape,f,f.sum(axis=1))
        cnn_model.classifier=torch.nn.Identity()
    else:
        base_model = load_model(base_model_name)
        layer_out = base_model.get_layer('global_pooling')
        cnn_model = Model(base_model.input,layer_out.output)

        #cnn_model.summary()
    
    train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
    test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
    train_features, train_data_files, test_features, test_data_files = extract_or_read_features(train_pickle, test_pickle, train_data_files, test_data_files)

Model:  enet_b0_8_va_mtl.pt
Create train dataset
Create test dataset


## Read features

In [15]:
base_model_key = 'enet_b0_8_best_afew.pt'
USERS_FEATURES = USERS_DATASET + 'features/'

In [34]:
engaged_pickle = USERS_DATASET + 'engaged_features_{}.extractor.pickle'.format(base_model_key)
eng_vid2features = load_features(engaged_pickle)

In [40]:
engaged_pickle_ref = USERS_FEATURES + 'engaged_features_{}.good.pickle'.format(base_model_key)
eng_vid2features_ref = load_features(engaged_pickle_ref)

In [41]:
print(eng_vid2features['Yana_engaged_2'])
print(eng_vid2features_ref['Yana_engaged_2'])
print(eng_vid2features['Yana_engaged_2']['features'] == eng_vid2features_ref['Yana_engaged_2']['features'])
print(eng_vid2features['Yana_engaged_2']['features'] - eng_vid2features_ref['Yana_engaged_2']['features'])
np.testing.assert_allclose(eng_vid2features['Yana_engaged_2']['features'], eng_vid2features_ref['Yana_engaged_2']['features'], rtol=1e-2, atol=1e-2)

{'features': array([[-0.17457296, -0.0838912 ,  0.0130755 , ..., -0.13524437,
        -0.19775935,  0.08375152],
       [-0.17530565, -0.09148189, -0.00201505, ..., -0.1188405 ,
        -0.20162667,  0.13017893],
       [-0.17212895, -0.09340156, -0.00343952, ..., -0.1270643 ,
        -0.2020545 ,  0.14086863],
       ...,
       [-0.15957235, -0.10684593, -0.04282501, ..., -0.12603702,
        -0.05989177,  0.14404123],
       [-0.12564267, -0.12438735, -0.04556637, ..., -0.1355438 ,
        -0.12425207,  0.16872056],
       [-0.09237796, -0.11252303, -0.07996154, ..., -0.17252916,
        -0.15970832,  0.07588568]], dtype=float32), 'label': 1}
{'features': array([[-0.17462769, -0.08404563,  0.01284902, ..., -0.13499106,
        -0.19764735,  0.08371742],
       [-0.17534773, -0.0915754 , -0.00221882, ..., -0.11862936,
        -0.20157237,  0.13044624],
       [-0.17213255, -0.09351983, -0.00365608, ..., -0.12657824,
        -0.2019372 ,  0.1411498 ],
       ...,
       [-0.15943566, 